In [1]:
import gymnasium as gym
from gymnasium.spaces import Discrete, Box, Sequence, Dict
import numpy as np
from src.commons import methods
import statistics

from src.rl_environments.env_impr import EnvImpr as Env

import os

## Config

In [2]:
DEFAULT_CONFIG = {
    'alpha_range': [ (0 + i / 10) for i in range(1,10)],
    'beta_range': [round(i * 0.5, 1) for i in range(2, 6)],
    'h_range': [0.5],
    'c_range': [25],
    'total': range(10, 40), 
}

TEST_CONFIG = {
    'alpha_range': [ (3 + i / 10) for i in range(1,10)],
    'beta_range': [round(i * 0.5, 1) for i in range(2, 6)],
    'h_range': [0.5],
    'c_range': [25],
    'total': range(10, 40), 
}



In [3]:
PPO_MODEL_NAME = 'PPO_Env1'

## Train PQN

In [4]:
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv

if True:

    n_cpus = os.cpu_count()

    print(f"Number of processors: {n_cpus}")
    env = make_vec_env(lambda: Env(config=DEFAULT_CONFIG), n_envs=n_cpus, vec_env_cls=SubprocVecEnv)
    model = PPO("MlpPolicy", env, verbose=1, device='cpu')

    print(model.n_envs)

    total_timesteps = 500_000
    print(f"Training model for {total_timesteps} timesteps")
    # Start training
    model.learn(total_timesteps=total_timesteps)

    model.save(methods.file_path(PPO_MODEL_NAME, 'models'))



Number of processors: 8
Using cpu device
8
Training model for 500000 timesteps
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 3.89     |
|    ep_rew_mean     | -3.91    |
| time/              |          |
|    fps             | 1032     |
|    iterations      | 1        |
|    time_elapsed    | 15       |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 4.72        |
|    ep_rew_mean          | -5.59       |
| time/                   |             |
|    fps                  | 960         |
|    iterations           | 2           |
|    time_elapsed         | 34          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.009490099 |
|    clip_fraction        | 0.0913      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38  

## Generate Test Data

In [5]:
def get_realized_data(config):
    alpha = np.random.choice(config['alpha_range'])
    beta = np.random.choice(config['beta_range'])
    h = np.random.choice(config['h_range'])
    c = np.random.choice(config['c_range'])
    total = np.random.choice(config['total'])
    intervals = np.random.gamma(shape=alpha, scale=beta, size=total)
    # travel_time = sum(intervals[4:]) - np.random.exponential(scale=alpha * beta)
    travel_time = sum(intervals[4:]) - np.random.gamma(shape=4, scale=alpha*beta)
    # travel_time = np.random.gamma(shape=5, scale=alpha*beta)
    # travel_time = max(beta * 5, travel_time)
    # travel_time = sum(intervals[3:]) * np.random.uniform(0, 1)
    travel_time = max(alpha * beta, travel_time)

    return alpha, beta, h, c, total, intervals, travel_time

import pandas as pd

df = pd.DataFrame(columns=['h', 'c', 'travel_time', 'total', 'intervals', 'alpha', 'beta'])

for i in range(10000):
    alpha, beta, h, c, total, intervals, travel_time = get_realized_data(TEST_CONFIG)
    row = {'h': h, 'c': c, 'travel_time': travel_time, 'total': total, 'intervals': intervals, 'alpha': alpha, 'beta': beta}
    df.loc[i] = row



## Test

In [6]:
from stable_baselines3 import PPO
model = PPO.load(methods.file_path(PPO_MODEL_NAME, 'models'))

env = Env(config=TEST_CONFIG)
rewards = {}
u_rl = {}
print(env._get_info())
for i in range(len(df)):
    row = df.iloc[i]
    state, _ = env.reset(row=row)
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(state)  
        state, reward, done, _, info = env.step(action)
        total_reward += reward
    rewards[i] = total_reward
    u_rl[i] = env.cur_time + env.travel_time
    # print(f"Episode {i} reward: {total_reward} | h = {info['state']['h']}, c = {info['state']['c']}")
    print(f"Episode {i} reward: {total_reward} | cur_time = {info['state']['cur_time']}")

df['rewards'] = rewards
df['u_rl'] = u_rl
print(f"Average reward: {df['rewards'].mean()}")

{'hidden': {'alpha': -1, 'beta': -1, 'interval': -1, 'cum_sum_intervals': -1}, 'state': {'n': -1, 'N': -1, 'travel_time': -1, 'cur_time': -1, 'mean_n': -1, 'std_n': -1, 'u_star_hat': -1, 'last_update': -1, 'remain_ratio': -1, 'scaled_diff_u_tt': -1, 'scaled_diff_last_update': -1}}
Episode 0 reward: -5.893706380878314 | cur_time = 57.48969587338374
Episode 1 reward: -7.705396033756607 | cur_time = 39.10718819144526
Episode 2 reward: -12.246943389880109 | cur_time = 28.73741654221489
Episode 3 reward: -13.635914313963639 | cur_time = 16.342599356170975
Episode 4 reward: -18.70727632632699 | cur_time = 20.451080200030567
Episode 5 reward: -16.339638202488473 | cur_time = 18.183190935024978
Episode 6 reward: -7.968258242586387 | cur_time = 39.502083533080324
Episode 7 reward: -5.573096231300639 | cur_time = 28.40418937766025
Episode 8 reward: -9.907354208986774 | cur_time = 26.547451845309645
Episode 9 reward: -12.188168691004563 | cur_time = 64.5824800703954
Episode 10 reward: -25 | cur_t

## Add Comparison Metrics

In [7]:

from pandarallel import pandarallel

# Initialize pandarallel with progress bar enabled
pandarallel.initialize(progress_bar=True)
for i in [0, 3, 5]:
    print(f"Optimal reward at n = {i}")
    df[f'u{i}'] = df.apply(lambda row: row['intervals'][i:].sum(), axis=1)
    df[f'u_star{i}'] = df.parallel_apply(lambda row: methods.get_u_star_binary_fast(row['total'] - i, row['alpha'], row['beta'], row['h'], row['c']), axis=1)
    df[f'optimal_rewards{i}'] = df.apply(lambda row: -methods.cal_cost(row['c'], row['h'], row[f'u{i}'], row[f'u_star{i}']), axis=1)

df['direct_leave_rewards'] = df.apply(lambda row: -methods.cal_cost(row['c'], row['h'], row['intervals'][3:].sum(), row[f'travel_time']), axis=1)
df.head()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Optimal reward at n = 0


Optimal reward at n = 3


Optimal reward at n = 5


,h,c,travel_time,total,intervals,alpha,beta,rewards,u_rl,u0,u_star0,optimal_rewards0,u3,u_star3,optimal_rewards3,u5,u_star5,optimal_rewards5,direct_leave_rewards
0,0.5,25,23.269234,12,"[7.19401951219961, 12.403804975411758, 16.1236...",3.1,2.0,-5.893706,80.758930,92.546343,59.820298,-16.363022,56.824829,42.127204,-7.348812,47.477775,30.611185,-8.433295,-16.777797
1,0.5,25,4.800000,13,"[3.3198018420898823, 4.697139896304993, 11.095...",3.2,1.5,-7.705396,43.907188,59.317980,49.468431,-4.924774,40.205137,35.980567,-2.112285,35.402640,27.180142,-4.111249,-17.702568
2,0.5,25,20.209196,10,"[8.594798674604801, 4.872499549419646, 4.87573...",3.2,2.5,-12.246943,48.946613,73.440500,64.431611,-4.504444,55.097468,41.413568,-6.841950,34.478982,26.581809,-3.948587,-17.444136
3,0.5,25,76.344031,23,"[4.317230077510242, 6.264159644598841, 3.42655...",3.5,1.5,-13.635914,92.686631,119.958459,105.880320,-7.039070,105.950517,90.474004,-7.738256,92.455556,80.262139,-6.096709,-14.803243
4,0.5,25,249.571421,36,"[6.1662190138786945, 10.896747599552551, 2.728...",3.8,2.5,-18.707276,270.022502,307.437054,330.114478,-25.000000,287.645686,300.746342,-25.000000,267.232323,281.199657,-25.000000,-19.037132


## Visualize Results

In [8]:
import plotly.graph_objects as go

all_rewards = [x for x in df.columns if 'reward' in x]

# Calculate means
means = df[all_rewards + ['h', 'c']].mean()

# Plot
fig = go.Figure(data=[go.Bar(x=means.index, y=means.values)])
fig.update_layout(title="Mean Values of Rewards, H, and C", yaxis_title="Mean Value")
fig.show()

# Calculate medians
medians = df[all_rewards + ['h', 'c']].median()

# Plot
fig = go.Figure(data=[go.Bar(x=medians.index, y=medians.values)])
fig.update_layout(title="Median Values of Rewards, H, and C", yaxis_title="Median Value")
fig.show()

In [9]:
df.head(50)

,h,c,travel_time,total,intervals,alpha,beta,rewards,u_rl,u0,u_star0,optimal_rewards0,u3,u_star3,optimal_rewards3,u5,u_star5,optimal_rewards5,direct_leave_rewards
0,0.5,25,23.269234,12,"[7.19401951219961, 12.403804975411758, 16.1236...",3.1,2.0,-5.893706,80.758930,92.546343,59.820298,-16.363022,56.824829,42.127204,-7.348812,47.477775,30.611185,-8.433295,-16.777797
1,0.5,25,4.800000,13,"[3.3198018420898823, 4.697139896304993, 11.095...",3.2,1.5,-7.705396,43.907188,59.317980,49.468431,-4.924774,40.205137,35.980567,-2.112285,35.402640,27.180142,-4.111249,-17.702568
2,0.5,25,20.209196,10,"[8.594798674604801, 4.872499549419646, 4.87573...",3.2,2.5,-12.246943,48.946613,73.440500,64.431611,-4.504444,55.097468,41.413568,-6.841950,34.478982,26.581809,-3.948587,-17.444136
3,0.5,25,76.344031,23,"[4.317230077510242, 6.264159644598841, 3.42655...",3.5,1.5,-13.635914,92.686631,119.958459,105.880320,-7.039070,105.950517,90.474004,-7.738256,92.455556,80.262139,-6.096709,-14.803243
4,0.5,25,249.571421,36,"[6.1662190138786945, 10.896747599552551, 2.728...",3.8,2.5,-18.707276,270.022502,307.437054,330.114478,-25.000000,287.645686,300.746342,-25.000000,267.232323,281.199657,-25.000000,-19.037132
5,0.5,25,147.889109,27,"[8.776255990945096, 2.406045012752231, 6.41433...",3.2,2.5,-16.339638,166.072300,198.751577,200.800154,-25.000000,181.154940,176.375214,-2.389863,166.927250,160.147825,-3.389713,-16.632915
6,0.5,25,12.839563,10,"[5.443377591440405, 8.133892163772927, 2.59211...",3.7,2.0,-7.968258,52.341646,68.278163,59.435914,-4.421124,52.108775,38.381788,-6.863493,35.208148,24.800489,-5.203829,-19.634606
7,0.5,25,31.162113,11,"[12.662445516234229, 9.129408530902499, 4.0301...",3.4,1.5,-5.573096,59.566302,70.712495,43.533217,-13.589639,44.890504,29.361701,-7.764402,33.307292,20.200918,-6.553187,-6.864196
8,0.5,25,217.319414,31,"[13.387446432588444, 5.657283870094614, 5.8435...",3.8,2.5,-9.907354,243.866866,263.681575,281.199657,-25.000000,238.793339,251.935836,-25.000000,226.671615,232.470026,-25.000000,-10.736962
9,0.5,25,217.485255,36,"[12.67980293769055, 9.124329148837559, 6.46228...",3.6,2.5,-12.188169,282.067735,306.444073,311.559844,-25.000000,278.177651,283.769999,-25.000000,259.519485,265.275249,-25.000000,-30.346198
